## Import LaTeX equations with text from csv

In [1]:
import pandas as pd
data = pd.read_csv("QUE4.csv")
# df = pd.DataFrame(data)
# df.tail()
# data.tail()
df = data.tail(5)

In [26]:
# type(data)
type(df)

pandas.core.frame.DataFrame

## Helper function to render image from LaTeX eqn. and save the image

In [2]:
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['mathtext.fontset'] = 'stix'

def render_latex_to_image(latex_string, filename):
    fig = plt.figure(figsize=(4,0.5))
    text = fig.text(x=0.5, y=0.5, s=latex_string, fontsize=11, ha='center', va='center')
    fig.savefig(filename, dpi=300, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)

In [3]:
latex_string = r"""$\underset{S}{\int\int}\ \vec{\nabla}\times\vec{B}\cdot d\vec{S}=\underset{C}{\oint}\ \vec{B}\cdot d\vec{l},$"""
render_latex_to_image(latex_string, 'outputxx.png')

## Output:
![](outputxx.png)

In [8]:
lat_string = r"""$\underset{S}{\int\int}\ \vec{\nabla}\times\vec{B}\cdot d\vec{S}=\underset{C}{\oint}\ \vec{B}\cdot d\vec{l},$"""
split_text = re.split(r'\$(.*?)\$', lat_string)
type(split_text)

list

## Helper function to wrap the text within the page

In [5]:
from reportlab.pdfbase.pdfmetrics import stringWidth

def wrap_text(text, max_width):
    words = text.split()
    lines = []
    current_line = words[0]
    for word in words[1:]:
        if stringWidth(current_line + " " + word, "Helvetica", 10) < max_width:
            current_line += " " + word
        else:
            lines.append(current_line)
            current_line = word
    lines.append(current_line)
    return lines

# Create a Directory to save LaTeX equation images

In [6]:
import os

latex_image_dir = "latex_images"
os.makedirs(latex_image_dir, exist_ok=True)

# Create PDF from csv

In [ ]:
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re

def create_pdf_with_latex(df, filename= "outputff.pdf"):
    
    #Set the canvas
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)

    # Layout Setting
    left_margin = 50
    top_margin = height - 50
    line_spacing = 20
    max_line_width = 575

    for index, row in df.iterrows():
        y_position = top_margin

        for col,value in row.items():
            text = f"{col}:{value}"
            lines = wrap_text(text, max_line_width)

            # Iterate for LateX Eqn.
            for line in lines:
                latex_match = re.findall(r'\$(.*?)\$', line)
                if latex_match:
                    x_position = left_margin
                    for eqn in latex_match:
                        eqn_pos = line.find(f"${eqn}$")
                        prefix_text = line[:eqn_pos]

                        # Draw text Upto the equation
                        c.drawString(x_position, y_position, prefix_text)
                        x_position += stringWidth(prefix_text, 'Helvetica', 12)

                        # Render Latex Equation and save as image file
                        latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                        if not os.path.exists(latex_image_path): # Avoid generating exixting image
                            wrapped_equation = f"""${eqn}$"""
                            render_latex_to_image(wrapped_equation, latex_image_path)

                        # Insert the saved Latex Image into the PDF
                        c.drawImage(latex_image_path, x_position + 40 , y_position - 10, width=70, height=30)

                        # Continue after equation
                        remaining_text = line[eqn_pos + len(eqn) + 2:]
                        next_pos = left_margin + stringWidth(prefix_text, 'Helvetica', 12) + 55 
                        c.drawString(next_pos, y_position, remaining_text)

                else:
                    # Draw Text Without Latex
                    c.drawString(left_margin, y_position, line)

                y_position -= line_spacing # Move for next line

            y_position -= line_spacing # Extra space after each row
        c.showPage()# For each row new page

    c.save()
    

create_pdf_with_latex(data)

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re

matplotlib.rcParams['mathtext.fontset'] = 'stix'
latex_image_dir = "latex_images"
os.makedirs(latex_image_dir, exist_ok=True)

data = pd.read_csv("QUE4.csv")
df = data.tail(5)  # Take the last 5 rows of the dataframe

def render_latex_to_image(latex_string, filename, img_width=50):
    fig = plt.figure(figsize=(img_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=11, ha='center', va='center')
    fig.savefig(filename, dpi=300, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)

def wrap_text(text, max_width):
    words = re.split(r'(\$.*?\$)', text)  # Split by LaTeX equations
    lines = []
    current_line = ""
    
    for word in words:
        if re.match(r'\$.*?\$', word):  # If it's a LaTeX equation
            if current_line:  # Add the current line if it has content
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)  # Treat the equation as a separate line
        else:  # Regular text
            if stringWidth(current_line + " " + word, "Helvetica", 12) < max_width:
                current_line += " " + word
            else:
                lines.append(current_line.strip())
                current_line = word
    if current_line:
        lines.append(current_line.strip())
    
    return lines

def create_pdf_with_latex(df, filename="output2fff.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)

    left_margin = 50
    top_margin = height - 50
    line_spacing = 20
    max_line_width = 575

    for index, row in df.iterrows():
        y_position = top_margin

        for col, value in row.items():
            text = f"{col}: {value}"
            lines = wrap_text(text, max_line_width)

            for line in lines:
                if re.match(r'\$.*?\$', line):  # If line is a LaTeX equation
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):  # Avoid regenerating existing image
                        render_latex_to_image(line, latex_image_path, img_width=70)

                    # Insert LaTeX image
                    c.drawImage(latex_image_path, left_margin, y_position - 10, width=70, height=30)
                    y_position -= line_spacing  # Move down after image
                else:
                    # Draw plain text
                    c.drawString(left_margin, y_position, line)
                    y_position -= line_spacing  # Move down for the next line

            y_position -= line_spacing  # Extra space after each row
        c.showPage()  # Start a new page for each row

    c.save()

create_pdf_with_latex(df)


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, landscape
import re
# Create a directory for LaTeX images
latex_img_dir = "latex_images"
os.makedirs(latex_img_dir, exist_ok=True)

# Function to render LaTeX equations to images and save to directory


# PDF generation function with LaTeX image insertion
def create_pdf_from_dataframe(df, filename="outputf.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)

    # Layout settings
    left_margin = 50
    top_margin = height - 50
    line_spacing = 20

    for index, row in df.iterrows():
        y_position = top_margin

        for col, value in row.items():
            text = f"{col}: {value}"

            # Process text and look for LaTeX equations
            latex_matches = re.findall(r'\$(.*?)\$', text)
            if latex_matches:
                split_text = re.split(r'\$(.*?)\$', text)  # Split text by LaTeX parts
                x_position = left_margin  # Start x-position for each row

                for i, segment in enumerate(split_text):
                    if i % 2 == 0:
                        # Draw non-LaTeX text
                        c.drawString(x_position, y_position, segment)
                        x_position += c.stringWidth(segment, "Helvetica", 12)
                    else:
                        # Render and insert LaTeX image
                        latex_img_path = os.path.join(latex_img_dir, f"latex_{index}_{i}.png")
                        latex_strings = f"""${''.join(segment)}$"""
                        render_latex_to_image(latex_strings, latex_img_path)
                        c.drawImage(latex_img_path, x_position, y_position - 10, width=50, height=20)
                        x_position += 50  # Adjust x-position based on image width

            else:
                # Draw plain text if no LaTeX
                c.drawString(left_margin, y_position, text)

            y_position -= line_spacing  # Move to the next line in the row

        c.showPage()  # Start a new page for each row

    c.save()  # Save the PDF

# Generate PDF with LaTeX-rendered images
create_pdf_from_dataframe(data)


In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re

matplotlib.rcParams['mathtext.fontset'] = 'stix'
latex_image_dir = "latex_images"
os.makedirs(latex_image_dir, exist_ok=True)

data = pd.read_csv("QUE4.csv")
df = data.tail(5)  # Take the last 5 rows of the dataframe

def render_latex_to_image(latex_string, filename, img_width=50):
    fig = plt.figure(figsize=(img_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=11, ha='center', va='center')
    fig.savefig(filename, dpi=300, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)

def wrap_text(text, max_width):
    words = re.split(r'(\$.*?\$)', text)  # Split by LaTeX equations
    lines = []
    current_line = ""
    
    for word in words:
        if re.match(r'\$.*?\$', word):  # If it's a LaTeX equation
            if current_line:  # Add the current line if it has content
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)  # Treat the equation as a separate line
        else:  # Regular text
            if stringWidth(current_line + " " + word, "Helvetica", 10) < max_width:
                current_line += " " + word
            else:
                lines.append(current_line.strip())
                current_line = word
    if current_line:
        lines.append(current_line.strip())
    
    return lines

def create_pdf_with_latex(df, filename="output3fff.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)

    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = 575

    for index, row in df.iterrows():
        y_position = top_margin

        for col, value in row.items():
            text = f"{col}: {value}"
            lines = wrap_text(text, max_line_width)
            x_position = left_margin  # Track position within line width

            for line in lines:
                if re.match(r'\$.*?\$', line):  # If line is a LaTeX equation
                    # Render LaTeX to image and calculate width
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        render_latex_to_image(line, latex_image_path, img_width=70)

                    image_width = 70
                    if x_position + image_width <= max_line_width:
                        # Draw image on the same line if space allows
                        c.drawImage(latex_image_path, x_position+40, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 45  # Adjust x-position
                    else:
                        # Start a new line if image doesn't fit
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 5

                else:
                    line_width = stringWidth(line, "Helvetica", 10)
                    if x_position + line_width <= max_line_width:
                        # Draw text on the same line if space allows
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5  # Adjust x-position
                    else:
                        # Start a new line if text doesn't fit
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing  # Extra space after each row
        c.showPage()  # Start a new page for each row

    c.save()

create_pdf_with_latex(df)


In [9]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re

# Set up for LaTeX rendering
matplotlib.rcParams['mathtext.fontset'] = 'cm'
latex_image_dir = "latex_images"
os.makedirs(latex_image_dir, exist_ok=True)

# Load data
data = pd.read_csv("QUE4.csv")
df = data.tail(5)

# Function to render LaTeX to image
def render_latex_to_image(latex_string, filename, img_width=50):
    fig = plt.figure(figsize=(img_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=11, ha='center', va='center')
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)

# Improved wrap_text function to handle long text lines
def wrap_text(text, max_width):
    words = re.split(r'(\$.*?\$)', text)  # Split by LaTeX equations
    lines = []
    current_line = ""

    for word in words:
        if re.match(r'\$.*?\$', word):  # LaTeX equation
            if current_line:  # Add current line to lines before LaTeX
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)  # Treat the equation as a separate line
        else:  # Regular text
            for sub_word in word.split():  # Check each sub-word separately
                if stringWidth(current_line + " " + sub_word, "Helvetica", 10) < max_width:
                    current_line += " " + sub_word
                else:
                    lines.append(current_line.strip())  # Add current line to lines
                    current_line = sub_word  # Start new line with current word
    if current_line:
        lines.append(current_line.strip())  # Add any remaining text
    
    return lines

# Function to create the PDF with LaTeX
def create_pdf_with_latex(df, filename="output_fixed_wrap_text.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)

    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = 575
    min_y_position = 50  # Ensure text doesn't go below this point

    for index, row in df.iterrows():
        y_position = top_margin

        for col, value in row.items():
            text = f"{col}: {value}"
            lines = wrap_text(text, max_line_width)
            x_position = left_margin

            for line in lines:
                # Ensure y_position stays within the page
                if y_position <= min_y_position:
                    c.showPage()
                    y_position = top_margin
                    x_position = left_margin

                if re.match(r'\$.*?\$', line):  # Line with LaTeX equation
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        render_latex_to_image(line, latex_image_path, img_width=70)

                    image_width = 70
                    if x_position + image_width <= max_line_width:
                        # Draw image if space allows
                        c.drawImage(latex_image_path, x_position + 40, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 45
                    else:
                        # Move to next line if image doesn't fit
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 5

                else:
                    line_width = stringWidth(line, "Helvetica", 10)
                    if x_position + line_width <= max_line_width:
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing  # Extra space after each row
        c.showPage()  # Start new page for each row

    c.save()

# Run function
create_pdf_with_latex(df)


In [11]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re

# Set up LaTeX rendering options for matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'  # Set font for LaTeX rendering
latex_image_dir = "latex_eqn_images"  # Directory to store generated images of LaTeX equations
os.makedirs(latex_image_dir, exist_ok=True)  # Ensure directory exists
# Register a custom font (e.g., OpenSans)
pdfmetrics.registerFont(TTFont("Lato", "/workspaces/codespaces-jupyter/fonts/Lato-Regular.ttf"))


# Load data from CSV file and select the last 5 rows
data = pd.read_csv("QUE4.csv")
# df = data.tail(5)

# Function to render LaTeX string as an image file
def render_latex_to_image(latex_string, filename, img_width=75):
    # Create a figure for rendering the LaTeX string
    fig = plt.figure(figsize=(img_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=14, ha='center', va='center')
    # Save the figure as an image with high DPI for clarity
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)  # Close the figure to free up resources

# Function to wrap text into lines that fit within a specified width
def wrap_text(text, max_width):
    # Split the text into parts, separating LaTeX equations and regular text
    words = re.split(r'(\$.*?\$)', text)
    lines = []
    current_line = ""

    for word in words:
        # If word is a LaTeX equation, add it as a separate line
        if re.match(r'\$.*?\$', word):
            if current_line:  # Append the current line if it contains text
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)  # Add the LaTeX equation as its own line
        else:  # If word is regular text, break it down into sub-words
            for sub_word in word.split():
                # Check if adding this word to the current line fits within max width
                if stringWidth(current_line + " " + sub_word, "Helvetica", 14) < max_width:
                    current_line += " " + sub_word
                else:
                    # If it doesn't fit, add the current line and start a new line
                    lines.append(current_line.strip())
                    current_line = sub_word  # Start new line with current word
    if current_line:  # Add any remaining text in current_line to lines
        lines.append(current_line.strip())
    
    return lines

# Function to generate a PDF with LaTeX-rendered images and wrapped text
def create_pdf_with_latex(df, filename="output_fixed_wrap_text_and_LaTeX_Eqn_increased_font.pdf"):

    c = canvas.Canvas(filename, pagesize=landscape(letter))  # Create PDF canvas
    width, height = landscape(letter)  # Get page dimensions
    c.setFont("Lato", 16)

    # Set layout margins and spacing parameters
    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = width - 2 * left_margin
    min_y_position = 50  # Minimum Y position to avoid text running off the page

    # Loop through each row in the dataframe
    for index, row in df.iterrows():
        y_position = top_margin  # Reset Y position for each row

        # Loop through each column and value in the row
        for col, value in row.items():
            text = f"{col}: {value}"  # Create text string for column and value
            lines = wrap_text(text, max_line_width)  # Wrap text to fit within max width
            x_position = left_margin  # Reset X position for each row

            # Loop through each wrapped line
            for line in lines:
                # Ensure y_position stays within the printable area
                if y_position <= min_y_position:
                    c.showPage()  # Start a new page if current Y position is too low
                    y_position = top_margin
                    x_position = left_margin

                if re.match(r'\$.*?\$', line):  # If the line contains a LaTeX equation
                    # Render LaTeX to image if not already done
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        render_latex_to_image(line, latex_image_path, img_width=70)

                    image_width = 70  # Define image width
                    if x_position + image_width <= max_line_width:
                        # Draw the image on the current line if it fits
                        c.drawImage(latex_image_path, x_position + 40, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 45
                    else:
                        # Move to the next line if image doesn't fit
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 5

                else:
                    # For regular text, measure the line width and position it accordingly
                    line_width = stringWidth(line, "Lato", 16)
                    if x_position + line_width <= max_line_width:
                        # Draw the text on the same line if it fits
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        # Move to the next line if text doesn't fit
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing  # Add extra space after each row
        c.showPage()  # Move to a new page after each row

    c.save()  # Save and close the PDF file

# Run the PDF generation function
create_pdf_with_latex(data)


Working with Custom Font
Error with Image resolution

In [15]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re

# Set up LaTeX rendering options for matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
pdfmetrics.registerFont(TTFont("Lato","fonts/Lato-Regular.ttf"))
latex_image_dir = "latex_eqn_images"
os.makedirs(latex_image_dir, exist_ok=True)

# Load data from CSV file and select the last 5 rows
data = pd.read_csv("linear_equations_questions.csv")
df = data.iloc[:,:-1]

# Function to render LaTeX string as an image file
def render_latex_to_image(latex_string, filename, base_img_width=30, scale_factor=1):
    equation_width = base_img_width + len(latex_string)
    fig = plt.figure(figsize=(equation_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=14, ha='center', va='center')
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)

# Function to wrap text into lines that fit within a specified width
def wrap_text(text, max_width):
    words = re.split(r'(\$.*?\$)', text)
    lines = []
    current_line = ""

    for word in words:
        if re.match(r'\$.*?\$', word):
            if current_line:
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)
        else:
            for sub_word in word.split():
                if stringWidth(current_line + " " + sub_word, "Lato", 16) < max_width:
                    current_line += " " + sub_word
                else:
                    lines.append(current_line.strip())
                    current_line = sub_word
    if current_line:
        lines.append(current_line.strip())
    
    return lines

# Function to draw a justified string
def draw_justified_string(c, text, x_position, y_position, max_line_width, font_name="Lato", font_size=16):
    words = text.split()
    space_width = stringWidth(" ", font_name, font_size)
    line_width = sum(stringWidth(word, font_name, font_size) for word in words) + space_width * (len(words) - 1)
    
    if line_width >= max_line_width or len(words) == 1:
        c.drawString(x_position, y_position, text)
        return

    extra_space = (max_line_width - line_width) / (len(words) - 1)
    x = x_position

    for word in words:
        c.drawString(x, y_position, word)
        x += stringWidth(word, font_name, font_size) + space_width + extra_space

# Main function to create PDF with justified text for 'question name' column
def create_pdf_with_latex(df, filename="output_fixed_wrap_text_and_LaTeX_Eqn_increased_font_linear_eqn.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)
    c.setFont("Lato", 16)

    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = width - 2 * left_margin
    min_y_position = 50

    for index, row in df.iterrows():
        y_position = top_margin

        for col, value in row.items():
            text = f"{col}: {value}"
            lines = wrap_text(text, max_line_width)
            x_position = left_margin

            for line in lines:
                if y_position <= min_y_position:
                    c.showPage()
                    y_position = top_margin
                    x_position = left_margin

                if col == "question name":
                    draw_justified_string(c, line, x_position, y_position, max_line_width)
                elif re.match(r'\$.*?\$', line):
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        render_latex_to_image(line, latex_image_path)

                    image_width, image_height = plt.imread(latex_image_path).shape[1],20
                    if x_position + image_width <= max_line_width:
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=image_height)
                        x_position += image_width + 5
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 5
                else:
                    line_width = stringWidth(line, "Lato", 16)
                    if x_position + line_width <= max_line_width:
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing
        c.showPage()
        c.setFont("Lato", 16) #Reset font for new Page 

    c.save()

# Run the function to create PDF
create_pdf_with_latex(df)


In [3]:
df = pd.read_csv("linear_equations_questions.csv")
df

,Question,Option A,Option B,Option C,Option D,Correct Option
0,$Solve x: 2x + 3 = 7$,x = 1,x = 2,x = 3,x = 4,B
1,$Solve x: 4x - 5 = 15$,x = 2.5,x = 3.5,x = 5,x = 7.5,C
2,$Solve x: 7x + 8 = 29$,x = 3,x = 4,x = 5,x = 6,A
3,$Solve x: x - 3 = 12$,x = 10,x = 12,x = 15,x = 9,C
4,A number increased by 5 is 12. Find the number.,5,6,7,8,B
5,"Three times a number, decreased by 4, equals 2...",x = 8,x = 7,x = 6,x = 9,A
6,The sum of a number and 10 is twice the number...,x = 5,x = 6,x = 7,x = 10,A
7,$Solve x: 3x + 5 = 11$,x = 1,x = 2,x = 3,x = 4,B
8,$5x - 7 = 13$,x = 4,x = 5,x = 6,x = 7,C
9,$Solve x: \frac{2}{3}x + 4 = 10$,x = 9,x = 8,x = 7,x = 6,B


In [4]:
df = df.iloc[:, :-1]
df

,Question,Option A,Option B,Option C,Option D
0,$Solve x: 2x + 3 = 7$,x = 1,x = 2,x = 3,x = 4
1,$Solve x: 4x - 5 = 15$,x = 2.5,x = 3.5,x = 5,x = 7.5
2,$Solve x: 7x + 8 = 29$,x = 3,x = 4,x = 5,x = 6
3,$Solve x: x - 3 = 12$,x = 10,x = 12,x = 15,x = 9
4,A number increased by 5 is 12. Find the number.,5,6,7,8
5,"Three times a number, decreased by 4, equals 2...",x = 8,x = 7,x = 6,x = 9
6,The sum of a number and 10 is twice the number...,x = 5,x = 6,x = 7,x = 10
7,$Solve x: 3x + 5 = 11$,x = 1,x = 2,x = 3,x = 4
8,$5x - 7 = 13$,x = 4,x = 5,x = 6,x = 7
9,$Solve x: \frac{2}{3}x + 4 = 10$,x = 9,x = 8,x = 7,x = 6


In [5]:
create_pdf_with_latex(df)

working


In [29]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re

# Set up LaTeX rendering options for matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
pdfmetrics.registerFont(TTFont("Lato","fonts/Lato-Regular.ttf"))
latex_image_dir = "latex_eqn_images_1"
os.makedirs(latex_image_dir, exist_ok=True)

# Load data from CSV file and select the last 5 rows
df = pd.read_csv("linears2.csv")
data = df.iloc[:, :-1]

# Function to render LaTeX string as an image file
def render_latex_to_image(latex_string, filename, base_img_width=100):
    img_width = base_img_width + len(latex_string) * 2
    fig = plt.figure(figsize=(img_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=14, ha='center', va='center')
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)

# Function to wrap text into lines that fit within a specified width
def wrap_text(text, max_width):
    words = re.split(r'(\$.*?\$)', text)
    lines = []
    current_line = ""

    for word in words:
        if re.match(r'\$.*?\$', word):
            if current_line:
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)
        else:
            for sub_word in word.split():
                if stringWidth(current_line + " " + sub_word, "Lato", 16) < max_width:
                    current_line += " " + sub_word
                else:
                    lines.append(current_line.strip())
                    current_line = sub_word
    if current_line:
        lines.append(current_line.strip())
    
    return lines

# Function to draw a justified string
def draw_justified_string(c, text, x_position, y_position, max_line_width, font_name="Lato", font_size=16):
    words = text.split()
    space_width = stringWidth(" ", font_name, font_size)
    line_width = sum(stringWidth(word, font_name, font_size) for word in words) + space_width * (len(words) - 1)
    
    if line_width >= max_line_width or len(words) == 1:
        c.drawString(x_position, y_position, text)
        return

    extra_space = (max_line_width - line_width) / (len(words) - 1)
    x = x_position

    for word in words:
        c.drawString(x, y_position, word)
        x += stringWidth(word, font_name, font_size) + space_width + extra_space

# Main function to create PDF with justified text for 'question name' column
def create_pdf_with_latex(df, filename="output_fixed_wrap_text_and_LaTeX_Eqn_increased_font_lin_eqn_test.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)
    c.setFont("Lato", 16)

    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = width - 2 * left_margin
    min_y_position = 50

    for index, row in df.iterrows():
        y_position = top_margin

        for col, value in row.items():
            text = f"{col}: {value}"
            lines = wrap_text(text, max_line_width)
            x_position = left_margin

            for line in lines:
                if y_position <= min_y_position:
                    c.showPage()
                    y_position = top_margin
                    x_position = left_margin

                if col == "question name":
                    draw_justified_string(c, line, x_position, y_position, max_line_width)
                elif re.match(r'\$.*?\$', line):
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        render_latex_to_image(line, latex_image_path)

                    image_width = 70
                    if x_position + image_width <= max_line_width:
                        c.drawImage(latex_image_path, x_position + 40, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 45
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 5
                else:
                    line_width = stringWidth(line, "Lato", 16)
                    if x_position + line_width <= max_line_width:
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing
        c.showPage()
        c.setFont("Lato", 16) #Reset font for new Page 

    c.save()

# Run the function to create PDF
create_pdf_with_latex(data)


Solve $x: 2x + 3 = \frac{x-1}{2}$


Solve $x: 8x - 6 = 3(2x + 1)$

Solve $x: \frac{3x +2}{4} = 2$

Solve  $x : \frac{5x3}{2} = x + 4$

Solve $x : 7x + 5 = \frac{4x + 1}{2}$

Solve $x: 4(x - 2) = 2(3x + 1)$

Solve $x: \frac{2x - 5}{3} - \frac{3x +2}{4} = \frac{x - 1}{2}$

Solve $x: \frac{x + 2}{5} + \frac{x -3}{2} = 4 - \frac{x+1}{3}$